# Speech-to-Text — Whisper Tiny ASR
**TL;DR:** Transcribe short audio clips with Whisper Tiny on CPU and compare with references.

**Models & Datasets:** [openai/whisper-tiny](https://huggingface.co/openai/whisper-tiny) (MIT), [LibriSpeech Dummy (toy)](https://huggingface.co/datasets/hf-internal-testing/librispeech_asr_dummy) (CC BY 4.0)
**Run Profiles:** 🖥️ CPU | 🍎 Metal (Apple Silicon) | 🧪 Colab/T4 | ⚡ CUDA GPU
**Env (minimal):** python>=3.10, transformers, datasets, evaluate, accelerate (optional: peft, bitsandbytes, timm, diffusers)
**Colab:** [Open in Colab](https://colab.research.google.com/github/SSusantAchary/Hands-On-Huggingface-AI-Models/blob/main/notebooks/audio/asr-whisper-tiny-base_cpu-first.ipynb)

**Switches (edit in one place):**
- `device` = {"cpu","mps","cuda"}
- `precision` = {"fp32","fp16","bf16","int8","4bit"}  (apply only if supported)
- `context_len` / `image_res` / `batch_size`

**Footprint & Speed (fill after run):**
- Peak RAM: TODO
- Peak VRAM: TODO (if GPU)
- TTFB: TODO, Throughput: TODO, Load time: TODO

**Gotchas:** Colab CUDA wheels may mismatch; pin torch/whisper versions ([Fixes & Tips](../fixes-and-tips/colab-cuda-mismatch.md))



## Setup
Load a dummy LibriSpeech slice and prepare the Whisper pipeline.


In [ ]:

import json
import os
import subprocess
import time
from pathlib import Path

import numpy as np
import torch
from datasets import load_dataset, Audio
from evaluate import load as load_metric
from transformers import pipeline

from notebooks._templates.measure import append_benchmark_row, measure_memory_speed

DEVICE_PREFERENCE = os.environ.get("HF_DEVICE", "cpu")
PRECISION = os.environ.get("HF_PRECISION", "fp32")

def resolve_device(preference: str = "cpu") -> str:
    if preference == "cuda" and torch.cuda.is_available():
        return "cuda:0"
    if preference == "mps" and torch.backends.mps.is_available():
        return "mps"
    return "cpu"

DEVICE = resolve_device(DEVICE_PREFERENCE)
print(f"Using device={DEVICE}")

MODEL_ID = "openai/whisper-tiny"
OUTPUT_DIR = Path("outputs") / "whisper-tiny"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

dataset = load_dataset(
    "hf-internal-testing/librispeech_asr_dummy",
    "clean",
    split="validation[:4]",
)
dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))


## Transcribe


In [ ]:

torch.manual_seed(42)

load_start = time.perf_counter()
asr = pipeline(
    "automatic-speech-recognition",
    model=MODEL_ID,
    device=DEVICE,
)
load_time = time.perf_counter() - load_start

transcripts = []
for row in dataset:
    audio = row["audio"]
    pred = asr({"array": audio["array"], "sampling_rate": audio["sampling_rate"]})
    transcripts.append(
        {
            "id": row["id"],
            "reference": row["text"],
            "prediction": pred["text"],
        }
    )

wer_metric = load_metric("wer")
wer = wer_metric.compute(
    predictions=[row["prediction"].lower() for row in transcripts],
    references=[row["reference"].lower() for row in transcripts],
)
print(f"WER: {wer:.3f}")

with open(OUTPUT_DIR / "transcripts.json", "w", encoding="utf-8") as fp:
    json.dump(transcripts, fp, indent=2)


## Measurement


In [ ]:

audio_batches = [
    {"array": row["audio"]["array"], "sampling_rate": row["audio"]["sampling_rate"]}
    for row in dataset
]

def run_inference(recorder):
    for idx, audio in enumerate(audio_batches):
        result = asr(audio)
        if idx == 0:
            recorder.mark_first_token()
        recorder.add_items(len(result["text"].split()))

metrics = measure_memory_speed(run_inference)

def fmt(value, digits=4):
    if value in (None, "", float("inf")):
        return ""
    return f"{value:.{digits}f}"

try:
    repo_commit = subprocess.check_output(["git", "rev-parse", "HEAD"], text=True).strip()
except Exception:  # noqa: BLE001
    repo_commit = ""

append_benchmark_row(
    task="whisper-asr",
    model_id=MODEL_ID,
    dataset="hf-internal-testing/librispeech_asr_dummy",
    sequence_or_image_res="15s@16kHz",
    batch=str(len(audio_batches)),
    peak_ram_mb=fmt(metrics.get("peak_ram_mb"), 2),
    peak_vram_mb=fmt(metrics.get("peak_vram_mb"), 2),
    load_time_s=fmt(load_time, 2),
    ttfb_s=fmt(metrics.get("ttfb_s"), 3),
    tokens_per_s_or_images_per_s=fmt(metrics.get("throughput_per_s"), 3),
    precision=PRECISION,
    notebook_path="notebooks/audio/asr-whisper-tiny-base_cpu-first.ipynb",
    repo_commit=repo_commit,
)

with open(OUTPUT_DIR / "metrics.json", "w", encoding="utf-8") as fp:
    json.dump({"wer": wer, **metrics}, fp, indent=2)
metrics


## Results Summary
        - Observations: TODO
        - Metrics captured: see `benchmarks/matrix.csv`

        ## Next Steps
        - TODOs: fill in after benchmarking

        ## Repro
        - Seed: 42 (set in measurement cell)
        - Libraries: captured via `detect_env()`
        - Notebook path: `notebooks/audio/asr-whisper-tiny-base_cpu-first.ipynb`
        - Latest commit: populated automatically when appending benchmarks (if git available)
